In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import string
import numpy as np
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
def google_search(x,n):
    driver = webdriver.Chrome('chromedriver')
    driver.get(f'https://www.google.com/search?q={x}')
    next_page_times = n
    http=[]
    for _page in range(next_page_times):
    
        soup = BeautifulSoup(driver.page_source,'html.parser')
        items=soup.find_all('div', attrs={'class':'yuRUbf'})

        for i in items:
            # 網址
            h=i.find('a')['href']
            if 'cdc.gov.tw' in h :
                pass
            else:
                http.append(h)
        # Wait
        time.sleep(random.uniform(1, 5))

        # Turn to the next page
        try:
            driver.find_element_by_link_text('下一頁').click()
        except:
            print('Search Early Stopping.')
            break
    driver.close()
    return http

In [3]:
def count_word(x):
    chinese=[]
    english=[]
    number=[]
    en=''
    num=''
    for i in list(x):

        if i in string.ascii_letters:
            en+=i
        elif i.isdigit():
            num+=i
        else:
            if en !='':
                english.append(en)
            if num!='':
                number.append(num)
            en=''
            num=''
            if i !=' ':
                chinese.append(i)
    words=len(english)+len(chinese)+len(number)
    return words

In [4]:
def apple_daily(x):
    apple2 = BeautifulSoup(x.text, 'html.parser')
    #標題
    title='｜'.join(apple2.find('title').text.split('｜')[:-2])
    try:
        title=title.replace('\u3000',' ')
    except:
        pass
    apple_text=apple2.select('p[class="text--desktop text--mobile article-text-size_md tw-max_width"]')
    
    content=''
    #內容
    try:
        for i in range(0,len(apple_text)):
            content+=apple2.select('p[class="text--desktop text--mobile article-text-size_md tw-max_width"]')[i].text
        try:
            content=content.replace('\u3000',' ')
        except:
            pass
    except:
        content=''
    #記者/機構
    try:
        a1=content.rfind('（')+1
        a2=content.rfind('╱')
        a3=content.rfind('／')
        a4=content.find('【')+1
        a5=content.find('╱')
        a6=content.find('／')
        if content[0]=='【':
            if a5>a6:
                author=content[a4:a5]
            else:
                author=content[a4:a6]
        else:
            if a2>a3:
                author=content[a1:a2]
            else:
                author=content[a1:a3]
    except:
        author=''
    #字數
    words=count_word(content)
    try:
        date=apple2.find('script',type="application/ld+json").string
        n=date.find('articleSection')
        #分類
        sention=date[n+17]+date[n+18]
        #日期
        k2=date.find('dateCreated')
        dates=date[k2+14]+date[k2+15]+date[k2+16]+date[k2+17]+date[k2+18]+date[k2+19]+date[k2+20]+date[k2+21]+date[k2+22]+date[k2+23]
    except:
        sention='None'
        dates='None'
    #圖片
    try:
        n_p=len(apple2.find_all('div',attrs={"style":"cursor:pointer"}))+len(apple2.find_all('div',attrs={"class":"promo-image-box"}))
    except:
        n_p='None'
    #影片
    try:
        v_p=len(apple2.find_all('div',attrs={'class':'promoItem'}))
    except:
        v_p='None'
    if v_p !=0:
        all_p=f'{n_p}（+{v_p}影片）'
    else:
        all_p=n_p
    if '疫苗' in content :
        return ['','',author,'',sention,dates,title,content,'','','','','',words,'',all_p,'']
    if words==1:
        return ['','',author,'',sention,dates,title,content,'','','','','',words,'',all_p,'']
    else:
        return['','','','','','','','','','','','','','','','','']

In [5]:
def web_crawl(x,n):
    http=google_search(x,n)
    apple=[]
    for i in range(0,len(http)):
        apple.append(requests.get(http[i]))
        time.sleep(random.uniform(1, 5))
    apple_result=list(map(apple_daily,apple))
    apple_df=pd.DataFrame(apple_result)
    apple_df=apple_df.rename(index=str,columns={0:'',1:'',2:'作者',3:'',4:'記者路線',5:'日期',6:'標題',7:'內文',8:'',9:'',10:'',11:'',12:'',13:'字數',14:'',15:"附圖（幀）",16:''})
    apple_df['網址']=http
    apple_df=apple_df[apple_df.標題!='']
    return apple_df

In [23]:
#k=[['疫苗 after:2021-06-09 before:2021-06-14 site:tw.appledaily.com',30],
 #['疫苗 after:2021-06-15 before:2021-06-18 site:tw.appledaily.com',30],
 #['疫苗 after:2021-06-19 before:2021-06-21 site:tw.appledaily.com',18]]
#dfs=[]
#for i in k:
    #df2=web_crawl(i[0],i[1])
    #dfs.append(df2)

In [41]:
#df=dfs[2]

In [6]:
df=web_crawl('疫苗 after:2021-07-06 before:2021-07-10 site:tw.appledaily.com',23)

In [7]:
df

,,,作者,,記者路線,日期,標題,內文,,,,,,字數,,附圖（幀）,,網址
0,,,突發中心陳韋劭、黃仲明,,社會,2021-07-07,關島打疫苗實錄｜1家4人如願接種 值得！飯店內2小時打完再看海景,關島開放國際旅客施打疫苗，台灣首發團前天（6日）就有164名旅客，充滿期待地搭乘長榮航空飛往...,,,,,,1408,,12（+1影片）,,https://tw.appledaily.com/local/20210708/7HDID...
1,,,即時新聞中心,,生活,2021-07-10,怒轟老人挑疫苗「不想打就別擋路」 醫師看到一亂象：很可怕,不少長者對AZ疫苗感到卻步，台北市立聯合醫院忠孝院區醫師陳昶宇看不下去，施打率「75歲以上長...,,,,,,998,,3,,https://tw.appledaily.com/life/20210710/NUUQMO...
2,,,過去有不少民眾，看似對於疫苗AZ的保護效力都有些遲疑，但有醫護公布接種完AZ疫苗後的結果，表...,,生活,2021-07-09,AZ有效嗎？禾馨公開員工打疫苗三周後檢驗報告「超驚人」,過去有不少民眾，看似對於疫苗AZ的保護效力都有些遲疑，但有醫護公布接種完AZ疫苗後的結果，表...,,,,,,452,,1（+1影片）,,https://tw.appledaily.com/life/20210709/ES7Y77...
3,,,沈能元、許稚佳、李姿慧,,生活,2021-07-09,增32例12死！ 陳時中：疫苗預約開放全國第九、十類已破200萬人,中央流行疫情指揮中心昨(7/8)宣布全國三級警戒延長至7月26日，但7月13日起微解封，不過...,,,,,,2108,,5（+1影片）,,https://tw.appledaily.com/life/20210709/P2JD5D...
4,,,黃仲丘,,生活,2021-07-06,疫苗何時輪到你一表掌握！郭台銘台積電BNT將到位 40歲有望10月接種,(增：指揮中心說法)國內新冠肺炎疫苗開放第一類到第八類人員優先施打後，總算要進入大量施打階段...,,,,,,1653,,5（+1影片）,,https://tw.appledaily.com/life/20210707/LGCDNA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,,,張哲鳴,,娛樂,2021-07-09,打完第2劑暈頭撞桌！徐若瑄跟兒子告假2天 傳授新加坡解燒祕方靠1種水果,徐若瑄（Vivian）在新加坡剛完成第2劑的疫苗，但她似乎有點擔心第2劑打完的反應。她打完在...,,,,,,1040,,2（+1影片）,,https://tw.appledaily.com/entertainment/202107...
225,,,政治中心/台北報導,,政治,2021-07-08,立陶宛10月來台設辦 學者：兩國有類似情感！未來有更多國家願跟進,波羅的海國家立陶宛多次公開表達將在台灣設立辦事處，經濟部長阿爾莫奈特接受德國媒體訪問時再度透...,,,,,,488,,1,,https://tw.appledaily.com/politics/20210709/4F...
227,,,地方中心陳宏銘,,生活,2021-07-09,屏東Delta退散！枋寮醫院拚復診 明後2日三度採檢是關鍵,屏東縣疫苗施打作業明天10日起至13日，將為65歲至68歲(原住民55至58歲)長者施打，一...,,,,,,931,,4,,https://tw.appledaily.com/life/20210709/CRZAIT...
228,,,許敏溶,,生活,2021-07-07,環南市場疫情已控制 公衛專家提出符合7個條件才能降級或微解,台灣近期本土疫情趨緩，民眾莫不盼望現有三級警戒能降級或微解封。公衛專家指出，北市環南市場群聚...,,,,,,1539,,1,,https://tw.appledaily.com/life/20210707/XA2JU2...


In [8]:
df2=df.drop_duplicates()

In [9]:
len(df2)

220

In [10]:
df2

,,,作者,,記者路線,日期,標題,內文,,,,,,字數,,附圖（幀）,,網址
0,,,突發中心陳韋劭、黃仲明,,社會,2021-07-07,關島打疫苗實錄｜1家4人如願接種 值得！飯店內2小時打完再看海景,關島開放國際旅客施打疫苗，台灣首發團前天（6日）就有164名旅客，充滿期待地搭乘長榮航空飛往...,,,,,,1408,,12（+1影片）,,https://tw.appledaily.com/local/20210708/7HDID...
1,,,即時新聞中心,,生活,2021-07-10,怒轟老人挑疫苗「不想打就別擋路」 醫師看到一亂象：很可怕,不少長者對AZ疫苗感到卻步，台北市立聯合醫院忠孝院區醫師陳昶宇看不下去，施打率「75歲以上長...,,,,,,998,,3,,https://tw.appledaily.com/life/20210710/NUUQMO...
2,,,過去有不少民眾，看似對於疫苗AZ的保護效力都有些遲疑，但有醫護公布接種完AZ疫苗後的結果，表...,,生活,2021-07-09,AZ有效嗎？禾馨公開員工打疫苗三周後檢驗報告「超驚人」,過去有不少民眾，看似對於疫苗AZ的保護效力都有些遲疑，但有醫護公布接種完AZ疫苗後的結果，表...,,,,,,452,,1（+1影片）,,https://tw.appledaily.com/life/20210709/ES7Y77...
3,,,沈能元、許稚佳、李姿慧,,生活,2021-07-09,增32例12死！ 陳時中：疫苗預約開放全國第九、十類已破200萬人,中央流行疫情指揮中心昨(7/8)宣布全國三級警戒延長至7月26日，但7月13日起微解封，不過...,,,,,,2108,,5（+1影片）,,https://tw.appledaily.com/life/20210709/P2JD5D...
4,,,黃仲丘,,生活,2021-07-06,疫苗何時輪到你一表掌握！郭台銘台積電BNT將到位 40歲有望10月接種,(增：指揮中心說法)國內新冠肺炎疫苗開放第一類到第八類人員優先施打後，總算要進入大量施打階段...,,,,,,1653,,5（+1影片）,,https://tw.appledaily.com/life/20210707/LGCDNA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,,,張哲鳴,,娛樂,2021-07-09,打完第2劑暈頭撞桌！徐若瑄跟兒子告假2天 傳授新加坡解燒祕方靠1種水果,徐若瑄（Vivian）在新加坡剛完成第2劑的疫苗，但她似乎有點擔心第2劑打完的反應。她打完在...,,,,,,1040,,2（+1影片）,,https://tw.appledaily.com/entertainment/202107...
225,,,政治中心/台北報導,,政治,2021-07-08,立陶宛10月來台設辦 學者：兩國有類似情感！未來有更多國家願跟進,波羅的海國家立陶宛多次公開表達將在台灣設立辦事處，經濟部長阿爾莫奈特接受德國媒體訪問時再度透...,,,,,,488,,1,,https://tw.appledaily.com/politics/20210709/4F...
227,,,地方中心陳宏銘,,生活,2021-07-09,屏東Delta退散！枋寮醫院拚復診 明後2日三度採檢是關鍵,屏東縣疫苗施打作業明天10日起至13日，將為65歲至68歲(原住民55至58歲)長者施打，一...,,,,,,931,,4,,https://tw.appledaily.com/life/20210709/CRZAIT...
228,,,許敏溶,,生活,2021-07-07,環南市場疫情已控制 公衛專家提出符合7個條件才能降級或微解,台灣近期本土疫情趨緩，民眾莫不盼望現有三級警戒能降級或微解封。公衛專家指出，北市環南市場群聚...,,,,,,1539,,1,,https://tw.appledaily.com/life/20210707/XA2JU2...


In [11]:
df2.to_excel('蘋果0706-0710.xlsx')

In [32]:
r=[]
for i in range(0,len(web_list)):
    r1 = requests.get(web_list[i])
    r.append(r1)
    time.sleep(random.uniform(1, 5))

In [7]:

df=pd.DataFrame()

In [16]:
r1

<Response [200]>

In [87]:
#print(soup.prettify())

In [24]:
r

[<Response [200]>]

In [33]:
title=[]
http=[]

In [34]:
for i in range(0,len(r)):
    if r[0].status_code == requests.codes.ok:
      # 以 BeautifulSoup 解析 HTML 原始碼
        soup = BeautifulSoup(r[0].text, 'html.parser')

      #觀察 HTML 原始碼
      # print(soup.prettify())

      # 以 CSS 的選擇器來抓取 Google 的搜尋結果
        items = soup.select('a[href^="/url"]')

        items= items[:-2]
        for i in items:
        # 標題
            title.append(i.text)
        # 網址
            http.append(i.get('href')[7:].split('&')[0])

In [35]:
dic={"標題":title,"網址":http}
df=pd.DataFrame(dic)

In [165]:
#print(soup.prettify())

In [176]:
df.to_excel('d.xlsx')

In [36]:
df

,標題,網址
0,國產疫苗大躍進！高端：第二期人體臨床試驗下周開打最快今年6月 ...tw.appledail...,https://tw.appledaily.com/life/20210106/AJOSWG...
1,【武漢肺炎】國際調查：41%台灣人對打疫苗有信心泰國接種疫苗 ...tw.appledail...,https://tw.appledaily.com/international/202101...
2,印度壯年男護士接種國產武肺疫苗翌日亡家屬質疑副作用所致 - 蘋果日報tw.appledail...,https://tw.appledaily.com/international/202101...
3,中國武肺疫苗副作用多達73種上海專家驚呼：世上最不安全｜ 蘋果 ...tw.appledai...,https://tw.appledaily.com/international/202101...
4,【武漢肺炎】中國科興疫苗有效性僅50% 官媒失焦：對重症完全保護 ...tw.appleda...,https://tw.appledaily.com/international/202101...
5,【1/11總疫報】英相：全國已完成240萬次疫苗接種密克羅尼西亞通報 ...tw.apple...,https://tw.appledaily.com/international/202101...
6,【1/15總疫報】挪威13名老人打疫苗後死亡歐洲確診總數逾3千萬 ...tw.appleda...,https://tw.appledaily.com/international/202101...
7,他接種武肺疫苗後生殖器增長7.6公分專家駁斥是謠言｜ 蘋果新聞網 ...tw.appleda...,https://tw.appledaily.com/international/202101...
8,日媒爆中共高層走私中國疫苗進日本中駐日使館表不滿｜ 蘋果新聞網 ...tw.appledai...,https://tw.appledaily.com/international/202101...
9,德國7人接種輝瑞武漢肺炎疫苗後死亡年齡介於79歲和93歲間 - 蘋果日報tw.appleda...,https://tw.appledaily.com/international/202101...


In [177]:
#print(soup2.prettify())

In [350]:
#print(soup.prettify())

In [37]:
apple=[]

In [38]:
for i in range(0,len(http)):
    apple.append(requests.get(http[i]))
    time.sleep(random.uniform(1, 5))

In [359]:
#apple=requests.get(http[0])

In [ ]:
def count_word(x):
    chinese=[]
    english=[]
    number=[]
    en=''
    num=''
    for i in list(x):

        if i in string.ascii_letters:
            en+=i
        elif i.isdigit():
            num+=i
        else:
            if en !='':
                english.append(en)
            if num!='':
                number.append(num)
            en=''
            num=''
            if i !=' ':
                chinese.append(i)
    words=len(english)+len(chinese)+len(number)
    return words

In [382]:
def apple_daily(x):
    apple2 = BeautifulSoup(x.text, 'html.parser')
    #標題
    title=apple2.find('title').text.split('｜')[0]
    try:
        title=title.replace('\u3000',' ')
    except:
        pass
    apple_text=apple2.select('p[class="text--desktop text--mobile article-text-size_md tw-max_width"]')
    
    content=''
    #內容
    for i in range(0,len(apple_text)):
        content+=apple2.select('p[class="text--desktop text--mobile article-text-size_md tw-max_width"]')[i].text
    try:
        content=content.replace('\u3000',' ')
    except:
        pass
    #字數
    words=count_word(content)
    date=apple2.find('script',type="application/ld+json").string
    n=date.find('articleSection')
    #分類
    sention=date[n+17]+date[n+18]
    #日期
    k2=date.find('dateCreated')
    dates=date[k2+14]+date[k2+15]+date[k2+16]+date[k2+17]+date[k2+18]+date[k2+19]+date[k2+20]+date[k2+21]+date[k2+22]+date[k2+23]
    return [title,content,words,sention,dates]

In [39]:
apple_result=list(map(apple_daily,apple))

In [40]:
apple_df=pd.DataFrame(apple_result)

In [41]:
apple_df

,0,1,2,3,4
0,國產疫苗大躍進！高端：第二期人體臨床試驗下周開打 最快今年6月全民接種,國際武漢肺炎疫情嚴峻，各國紛紛開打武肺疫苗，盼能阻止疫情擴散。我國武肺疫苗研發進度今（1/6...,1012,生活,2021-01-06
1,【武漢肺炎】國際調查：41%台灣人對打疫苗有信心 泰國接種疫苗意願最高,多國陸續批出不同藥企研發的武漢肺炎疫苗接種許可，不少疫苗也踏入最後階段臨床測試，民眾是否願意...,722,國際,2021-01-16
2,印度壯年男護士接種國產武肺疫苗翌日亡 家屬質疑副作用所致,印度上周六展開全球規模最大的武漢肺炎疫苗接種計劃，盼在7月底前為大約3億人接種。當地已有22...,831,國際,2021-01-18
3,中國武肺疫苗副作用多達73種 上海專家驚呼：世上最不安全,中國醫藥集團（Sinopharm，以下簡稱：國藥集團）研發的武漢肺炎滅活疫苗（不活化疫苗，由...,1907,國際,2021-01-05
4,【武漢肺炎】中國科興疫苗有效性僅50% 官媒失焦：對重症完全保護,在巴西進行臨床試驗的中國科興武漢肺炎疫苗整體有效性50.38%，達到世衛的最低標準，但相較其...,770,國際,2021-01-13
5,【1/11總疫報】英相：全國已完成240萬次疫苗接種 密克羅尼西亞通報首起病例,武漢肺炎自2019年12月由中國蔓延至全球後，截至今天（11日）目前，已造成超過9047萬人...,2565,國際,2021-01-10
6,【1/15總疫報】挪威13名老人打疫苗後死亡 歐洲確診總數逾3千萬,武漢肺炎自2019年12月由中國蔓延至全球後，截至今天（15日）目前，已造成超過9333萬人...,2878,國際,2021-01-14
7,他接種武肺疫苗後生殖器增長7.6公分 專家駁斥是謠言,中國武漢肺炎疫情延燒，民眾們陸續施打疫苗，然而近來中國網路流傳一名印尼男子在接種中國疫苗後，...,403,國際,2021-01-17
8,日媒爆中共高層走私中國疫苗進日本 中駐日使館表不滿,有日媒報導指出，一名與中共有密切關係的中國人士將疫苗走私到日本，提供給多位知名企業家在內的日...,889,國際,2021-01-02
9,德國7人接種輝瑞武漢肺炎疫苗後死亡 年齡介於79歲和93歲間,德國負責審查疫苗的官方機構保羅艾爾立希研究所，日前發布報告表示，該機構已統計到325例疑似與...,672,國際,2021-01-17


In [412]:
apple_df=apple_df.rename(index=str,columns={0:'標題',1:'內文',2:'字數',3:'分類',4:'日期'})

In [413]:
apple_df['網址']=http

In [414]:
apple_df

,標題,內文,字數,分類,日期,網址
0,【社區風暴】創單日新高！昨1.1萬人接種疫苗 車隊補助運將免費打,因為昨新增7例疫情促進疫苗「打氣」，中央流行疫情指揮中心統計，昨有1萬1018人打疫苗，當中...,814,生活,2021-05-12,https://tw.appledaily.com/life/20210512/VZCBVR...
1,國籍航空姐打疫苗驚傳副作用！ 眼睛血栓模糊控醫不敢通報,(新增：指揮中心回應)國籍航空服員傳出施打AZ疫苗後出現眼睛血栓、視力減退等嚴重不良反應，有...,2032,生活,2021-05-13,https://tw.appledaily.com/life/20210513/WZCK7N...
2,蔡英文為全台大停電致歉！「國產疫苗7月底上路 鼓勵國人踴躍預約登記」,新冠肺炎（COVID-19）國內疫情持續升溫，社區不明感染源陸續爆發，執政團隊繃緊神經應戰。...,1983,政治,2021-05-13,https://tw.appledaily.com/politics/20210513/6P...
3,台灣防疫隱憂 《彭博》曝疫苗接種率不到1％,台灣疫情已進入了社區感染。美國財經媒體彭博（Bloomberg）報導，指這是台灣自去年放寬防...,430,國際,2021-05-11,https://tw.appledaily.com/international/202105...
4,單日1.1萬人接種疫苗 創新高,國內接連爆發COVID-19本土病例，使疫苗打氣升溫。中央流行疫情指揮中心昨公布，前天有1萬...,459,要聞,2021-05-12,https://tw.appledaily.com/headline/20210513/4S...
5,65歲以上公費開打AZ疫苗 各縣市多家醫院可現場掛號接種,中央流行疫情指揮中心指揮官陳時中上周四（6日）宣布，開放第7、8類對象公費接種AZ疫苗，其中...,1509,生活,2021-05-10,https://tw.appledaily.com/life/20210510/DUTJYI...
6,【社區風暴】疫苗搶打潮出現 台中兩醫學中心打疫苗人數爆3倍,中央疫情指揮中心今天公布新冠肺炎本土病例一夕爆增16人，引發民眾恐慌，原本乏人問津的新冠肺炎...,523,生活,2021-05-12,https://tw.appledaily.com/life/20210512/M7A2PK...
7,【疫苗開打】逾5千人預約自費接種 陳時中：外籍人士比照國人收費,中央流行疫情指揮中心今(4/21)下午由指揮官陳時中召開例行疫情記者會。統計至昨（4/20）...,974,生活,2021-04-21,https://tw.appledaily.com/life/20210421/N6XRAR...
8,打新冠疫苗 可請2天無薪假,【生活中心／台北報導】為鼓勵民眾施打COVID-19疫苗，中央流行疫情指揮中心昨宣布，即起勞...,852,要聞,2021-05-05,https://tw.appledaily.com/headline/20210506/L7...
9,【疫苗來了】柯P可打疫苗了！AZ再到貨近20萬劑 4/12起官員、機組員、防疫小黃接種,台灣採購的牛津(AZ)疫苗已達31.6萬劑，昨天(4/6)擴大接種對象；歐盟即將開會討論牛津...,1623,生活,2021-04-04,https://tw.appledaily.com/life/20210407/TBPHQN...


In [27]:
apple_daily(requests.get('https://tw.appledaily.com/international/20210128/M4T66XK3FVGDHI3TA3OCRT7MIA/'))

['',
 '',
 '北美中心',
 '',
 '國際',
 '2021-01-27',
 '【武漢肺炎】加中疫苗合作告吹原因曝光：中國海關扣起樣本 ',
 '加拿大總理杜魯多（Justin Trudeau）去年5月中突然宣布，與中國康希諾公司研究武漢肺炎疫苗，並批准在當地開始臨床試驗。惟數日後即宣佈合作告吹，有關調查報告日前交予加拿大國會，顯示中國政府阻止疫苗樣本運送到加拿大。專家認為，這是中國為了孟晚舟一案向加拿大施壓。去年5月16日，杜魯多宣布聯邦衛生部門，已經與康希諾在戴爾豪斯大學（Dalhousie University）合作就疫苗開始臨床試驗，指「這是令人鼓舞的消息，研究和開發需要時間，必須好好地做」。但根據國會報告，中國海關在5月18日於北京首都國際機場，扣押一批運往加拿大的「Ad5-nCoV」疫苗樣本，直到8月下旬，加中兩國疫苗合作計劃被迫放棄，消息才被公開。加拿大國家研究委員會在回答保守黨議員提問時表示，疫苗特別工作組原本對康希諾疫苗抱有很大期望，但由於運送方面有所延誤，當局對康希諾的研究「失去熱情」。同時，隨著其他疫苗的研究進展良好，以及康希諾候選疫苗已在其他國家作臨床試驗，因此8月下旬停止讓康希諾候選疫苗在加拿大作臨床試驗，並專注於其他候選疫苗上。整場疫苗運送風波發生於去年5月，正好是華為副董事長孟晚舟，在加拿大溫哥華卑詩省最高法院被判「雙重犯罪」成立。渥太華大學科學﹑社會與政策研究所高級研究員約翰斯頓（Margaret McCuaig-Johnston）認為，事件責任在中方，「當我們與中國的接觸越多，他們就有更多機會增加壓力（turn the screws）」。加拿大《環球郵報》（The Globe and Mail）指，目前未能確認中方是否單方面阻止把疫苗樣本送到加拿大。加拿大保守黨指責杜魯多政府，過份依賴與康希諾公司合作生產疫苗，忽略了與輝瑞及莫德納討論疫苗交收進度。聯邦採購部長阿蘭德（Anita Anand）不認同有關指控，指康希諾項目是前加拿大創新科技及工業部長貝恩斯（Navdeep Bains）責任，但也不妨礙她與其他供應商談判。目前，加拿大在疫苗分發及管理上，遠遠落後其他國家，同時，因為輝瑞的生產力放緩，加拿大本周收不到任何疫苗，致未來一周僅能接種7.9萬劑。（北美中心／綜合外電報導）',
 '',
 '',
 '',
 '',

In [2]:
apple2 = BeautifulSoup(requests.get('https://tw.appledaily.com/life/20210218/U5YPGK653VAHVCETKAC34GPCMY/'
).text, 'html.parser')

In [9]:
apple3 = BeautifulSoup(requests.get('https://tw.appledaily.com/life/20210531/L5Z3D3XYQ5ABBERLZX2TVEMJUQ/'
).text, 'html.parser')

In [135]:
apple4 = BeautifulSoup(apple[2].text, 'html.parser')

In [134]:
apple5 = BeautifulSoup(apple[3].text, 'html.parser')

In [82]:
web_list

['https://www.google.com/search?q=%E7%96%AB%E8%8B%97+after%3A2021-01-01+before%3A2021-01-20+site%3Atw.appledaily.com&oq=&aqs=chrome.3.35i39i362l8...8.9334j0j7&sourceid=chrome&ie=UTF-8']

In [ ]:
apple2.

### 標題

In [24]:
title='｜'.join(apple2.find('title').text.split('｜')[:-2])
try:
    title=title.replace('\u3000',' ')
except:
    pass
print(title)

新北疫情｜醫護疫苗已打42.7％ 侯友宜喊話中央「完全不夠用」 


In [25]:
title='｜'.join(apple3.find('title').text.split('｜')[:-2])
try:
    title=title.replace('\u3000',' ')
except:
    pass
print(title)

「政治歸政治、科學歸科學太天真嗎？」國產疫苗5大疑問 計畫主持人解答 


In [23]:
'｜'.join(apple3.find('title').text.split('｜')[:-2])

'「政治歸政治、科學歸科學太天真嗎？」國產疫苗5大疑問\u3000計畫主持人解答 '

### 內文

In [20]:
apple_text=apple2.select('p[class="text--desktop text--mobile article-text-size_md tw-max_width"]')

In [21]:
content=''
for i in range(0,len(apple_text)):
    content+=apple2.select('p[class="text--desktop text--mobile article-text-size_md tw-max_width"]')[i].text
try:
    content=content.replace('\u3000',' ')
except:
    pass

In [22]:
content

'台灣採購的牛津(AZ)疫苗已達31.6萬劑，昨天(4/6)擴大接種對象；歐盟即將開會討論牛津(AZ)疫苗與血栓間的關係，世界衛生組織(WHO)仍呼籲施打疫苗為好處多於壞處，應要持續施打。中央流行疫情指揮中心今(4/7)14時由指揮官陳時中召開例行疫情記者會。陳時中宣布，4月12日起，開放中央及地方政府防疫人員及高接觸風險工作者等對象接種疫苗。陳時中表示，為維持防疫量能，使高感染風險對象儘早獲得免疫保護力，自4月12日起，開放中央及地方政府防疫人員及高接觸風險工作者等接種計畫所列第二、三類實施對象接種疫苗。此外，陳時中說，4月12日起擴大疫苗施打對象至第二類、第三類，第二類「中央及地方政府防疫人員」估計9萬人，第三類「高接觸風險工作者」約有3.5萬人，加上第一類所有醫事人員，總計61.3萬人納入31.6萬劑AZ疫苗的施打範圍。指揮中心表示，我國透過COVAX(COVID-19疫苗全球取得機制)獲配AZ疫苗，首批19.92萬劑已於4月4日運抵臺灣，目前正由衛生福利部食品藥物管理署進行檢驗封緘作業，預計4月中旬可放行提供接種。為加速提升群體免疫力，發揮疫苗最大使用效益，故自4月12日起，開放中央及地方政府防疫人員以及國籍航空機組員、國際商船船員、防疫車隊駕駛、防疫旅宿實際執行居家檢疫工作之第一線人員等高接觸風險工作者接種疫苗。指揮中心說明，為增加疫苗接種可近性，提升接種效能，將請地方政府衛生局與現行合約醫院持續溝通，增加疫苗接種門診診次及每診次就診名額，並於4月14日前輔導具有辦理常規或流感疫苗接種經驗，但尚未成為疫苗接種合約院所之專責醫院成為接種點，自4月15日起開放提供符合接種對象之民眾預約，預計自4月16日起，全臺至少將有170處接種點可提供接種服務，符合接種資格對象可至疾病管制署全球資訊網查詢院所後預約接種。由於疫苗為多劑型包裝，為減少疫苗耗損，請依預約時間，攜帶健保卡等身分證明文件準時前往接種。目前我國收到疫苗30多萬劑，開放到第三順位累計有61萬人可接種，目前是希望全打第一劑還是全需施打兩劑？陳時中透露自己的算法。他說，6月15日為疫苗有效期限，往回推8周，觀察接種狀況約到4月10幾號，屆時打多少劑就保留住幾劑，這樣到5月17日時，就能讓原先接種者可用保留的劑量打第二劑，中間就盡量開放給大家打，既可擴大施打人數、又能保證原先打過第一劑者可打到第二劑，如過程中

In [23]:
a1=content.rfind('（')+1
a2=content.rfind('╱')
content[a1:a2]

'沈能元、黃仲丘、林芳如、許稚佳／台北報導'

In [83]:
words=len(content)
print(words)

1963


In [100]:
#apple2.find_all('script',type="application/ld+json")

In [24]:
date=apple2.find('script',type="application/ld+json").string

In [25]:
date

'{"@context":"http://schema.org","@type":"NewsArticle","mainEntityOfPage":{"@type":"WebPage","@id":"https://tw.appledaily.com/life/20210407/TBPHQNS2CFBYRBTKCIZJXG4IDQ/"},"headline":"【疫苗來了】柯P可打疫苗了！AZ再到貨近20萬劑\u30004/12起官員、機組員、防疫小黃接種","image":{"@type":"ImageObject","url":"https://hk.appledaily.com/resizer/xNF_ArRwBUx33kI0uF55KqO-Lns=/filters:quality(100)/d87urpdhi5rdo.cloudfront.net/04-07-2021/t_caac3ab8e24d4291ba8ea7185fa46fea_name_t_5153349782f54bf5857f9ba798afda0b_name_08042021_vtnews_07____v2_____.jpg","siteProperties":{"siteName":"tw-appledaily","siteDomainURL":"https://tw.appledaily.com","sites":["hk-appledaily","tw-appledaily"],"siteBasePaths":{"hk-appledaily":{"basePath":"https://hk.appledaily.com","displayName":"香港","countryCode":"hk"},"tw-appledaily":{"basePath":"https://tw.appledaily.com","displayName":"台灣","countryCode":"tw"},"north-america":{"basePath":"/realtime/us/","displayName":"北美","countryCode":"na"},"dev":"https://hk.appledaily.com","sandbox":"http://appledaily-tw-appl

In [52]:
k=date.find('articleSection')
sention=date[k+17]+date[k+18]
print(sention)

生活


In [31]:
k2=date.find('dateCreated')
dates=date[k2+14]+date[k2+15]+date[k2+16]+date[k2+17]+date[k2+18]+date[k2+19]+date[k2+20]+date[k2+21]+date[k2+22]+date[k2+23]
print(dates)

2021-04-04


In [49]:
#apple2.find_all('div',attrs={"class":"image-container"})

In [134]:
#apple2.find_all('figure')

In [178]:
p_n=len(apple2.find_all('div',attrs={"style":"cursor:pointer"}))+len(apple2.find_all('div',attrs={"class":"promo-image-box"}))
v_n=len(apple2.find_all('div',attrs={'class':'promoItem'}))

In [185]:
print(f'{p_n}+({v_n}影片)')

1+(0影片)


In [139]:
#apple2.find_all('script',type="application/javascript")[4].string

In [140]:
#apple3.find_all('script',type="application/javascript")[4].string

In [141]:
#apple5.find_all('script',type="application/javascript")[4].string

In [142]:
#apple4.find_all('script',type="application/javascript")[4].string

In [109]:
apple2.find_all('figure')

[<figure><div class="image-container" style="background-color:gainsboro"><figure><img alt="missing promo" src="/pf/resources/images/icons/tw-appledaily/no_image.png?d=215" style="height:auto;width:100%"/></figure></div></figure>,
 <figure><img alt="missing promo" src="/pf/resources/images/icons/tw-appledaily/no_image.png?d=215" style="height:auto;width:100%"/></figure>,
 <figure><div class="image-container" style="background-color:gainsboro"><figure><img alt="missing promo" src="/pf/resources/images/icons/tw-appledaily/no_image.png?d=215" style="height:auto;width:100%"/></figure></div></figure>,
 <figure><img alt="missing promo" src="/pf/resources/images/icons/tw-appledaily/no_image.png?d=215" style="height:auto;width:100%"/></figure>,
 <figure><div class="image-container" style="background-color:gainsboro"><figure><img alt="missing promo" src="/pf/resources/images/icons/tw-appledaily/no_image.png?d=215" style="height:auto;width:100%"/></figure></div></figure>,
 <figure><img alt="missi

In [3]:
print(apple2.prettify())

<!DOCTYPE html>
<html lang="zh-tw">
 <head>
  <script type="application/javascript">
   if(!Array.prototype.includes||!(window.Object && window.Object.assign)||!window.Promise||!window.Symbol||!window.fetch){document.write('<script type="application/javascript" src="/pf/dist/engine/polyfill.js?d=239" defer=""><\/script>')}
  </script>
  <script defer="" id="fusion-engine-script" src="/pf/dist/engine/react.js?d=239" type="application/javascript">
  </script>
  <script defer="" id="fusion-engine-script" src="/pf/dist/components/combinations/default.js?d=239" type="application/javascript">
  </script>
  <link href="/pf/dist/components/combinations/default.css?d=239" id="fusion-template-styles" rel="stylesheet" type="text/css"/>
  <link href="https://hk.appledaily.com" hreflang="zh-hk" rel="alternate"/>
  <link href="https://tw.appledaily.com" hreflang="zh-tw" rel="alternate"/>
  <link href="https://hk.appledaily.com" hreflang="x-default" rel="alternate"/>
  <title>
   突破中國攔截！陳時中抱怨奏效　德國BNT

In [36]:
df['網址']

0      https://tw.appledaily.com/life/20210517/YWO7VL...
1      https://tw.appledaily.com/life/20210517/UP6JGE...
2      https://tw.appledaily.com/property/20210517/ZX...
3      https://tw.appledaily.com/life/20210517/DYCZSX...
4      https://tw.appledaily.com/life/20210517/MWIQ4L...
                             ...                        
280    https://tw.appledaily.com/life/20210531/EZOJJN...
281    https://tw.appledaily.com/life/20210531/6YECSS...
282    https://tw.appledaily.com/entertainment/202105...
283    https://tw.appledaily.com/politics/20210531/T7...
284    https://tw.appledaily.com/politics/20210531/CS...
Name: 網址, Length: 285, dtype: object

In [35]:
df=pd.read_excel('蘋果0517-0531.xlsx')

In [42]:
titles=[]
for i in df['網址']:
    apples= BeautifulSoup(requests.get(i).text, 'html.parser')
    time.sleep(random.uniform(1, 5))
    title2='｜'.join(apples.find('title').text.split('｜')[:-2])
    try:
        title2=title2.replace('\u3000',' ')
    except:
        pass
    titles.append(title2)

In [49]:
pd.DataFrame(titles,columns=['標題']).to_excel('標題補.xlsx')

In [44]:
titles.to_excel('標題補.xlsx')

AttributeError: 'list' object has no attribute 'to_excel'